<a href="https://colab.research.google.com/github/5dogs/data-science/blob/main/Customer_Personality_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')
# 基本操作
import pandas as pd
import numpy as np
from google.colab import drive
import os
import shutil

# 可視化
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

# ロジスティック回帰
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

# その他
import re
import datetime
import warnings
import random

# %matplotlib inline

In [3]:
# ファイルパス
# dir_path = "/content/drive/MyDrive/ColabNotebooks/Customer Personality Analysis/"
dir_path = "/content/"
file_name = "m_c_shorten.csv"

# データの読み込み
df = pd.read_csv(
    dir_path + file_name,
    sep='\t',                   # タブ区切りの指定
    parse_dates=['Dt_Customer'], # 'Dt_Customer'を日付型として読み込む
    index_col='ID',             # 'ID'をインデックスとして設定
    encoding='utf-8'            # エンコーディングの指定（必要に応じて調整）
)

# データの先頭5行を表示
print(df.head(1))

      Year_Birth   Education Marital_Status   Income  Kidhome  Teenhome  \
ID                                                                        
5524        1957  Graduation         Single  58138.0        0         0   

     Dt_Customer  Recency  MntWines  MntFruits  ...  NumWebVisitsMonth  \
ID                                              ...                      
5524  04-09-2012       58       635         88  ...                  7   

      AcceptedCmp3  AcceptedCmp4  AcceptedCmp5  AcceptedCmp1  AcceptedCmp2  \
ID                                                                           
5524             0             0             0             0             0   

      Complain  Z_CostContact  Z_Revenue  Response  
ID                                                  
5524         0              3         11         1  

[1 rows x 28 columns]


In [ ]:
# データ型の確認
dtype_sr = df.dtypes
print(dtype_sr)

# 欠損値の数
nullcnt_sr = df.isnull().sum()
print(nullcnt_sr)


Year_Birth                int64
Education                object
Marital_Status           object
Income                  float64
Kidhome                   int64
Teenhome                  int64
Dt_Customer              object
Recency                   int64
MntWines                  int64
MntFruits                 int64
MntMeatProducts           int64
MntFishProducts           int64
MntSweetProducts          int64
MntGoldProds              int64
NumDealsPurchases         int64
NumWebPurchases           int64
NumCatalogPurchases       int64
NumStorePurchases         int64
NumWebVisitsMonth         int64
AcceptedCmp3              int64
AcceptedCmp4              int64
AcceptedCmp5              int64
AcceptedCmp1              int64
AcceptedCmp2              int64
Complain                  int64
Z_CostContact             int64
Z_Revenue                 int64
Response                  int64
Age                       int64
Age_Group              category
dtype: object
Year_Birth             0
E

素のやつ  
データ全選択→png保存→pdf出力

In [ ]:
# prompt: 各インデックスをすべて可視化したものをすべてまとめて一つのファイルにし、ダウンロードしたい

import matplotlib.pyplot as plt
# 各列のヒストグラムをプロット
for col in df.columns:
  if df[col].dtype in ['int64', 'float64']:  # 数値データのみヒストグラムを作成
    plt.figure()
    df[col].hist()
    plt.xlabel(col)
    plt.ylabel('Frequency')
    plt.title(f'Histogram of {col}')
    plt.savefig(f'{col}_histogram.png')  # ヒストグラムを画像ファイルとして保存
    plt.close()

# 各列の箱ひげ図をプロット
for col in df.columns:
  if df[col].dtype in ['int64', 'float64']:  # 数値データのみ箱ひげ図を作成
    plt.figure()
    df.boxplot(column=col)
    plt.title(f'Boxplot of {col}')
    plt.savefig(f'{col}_boxplot.png')  # 箱ひげ図を画像ファイルとして保存
    plt.close()

# 各列の散布図をプロット（数値データ同士の組み合わせ）
numeric_cols = df.select_dtypes(include=['number']).columns
for i in range(len(numeric_cols)):
  for j in range(i + 1, len(numeric_cols)):
    col1 = numeric_cols[i]
    col2 = numeric_cols[j]
    plt.figure()
    plt.scatter(df[col1], df[col2])
    plt.xlabel(col1)
    plt.ylabel(col2)
    plt.title(f'Scatter plot of {col1} vs {col2}')
    plt.savefig(f'{col1}_vs_{col2}_scatter.png')  # 散布図を画像ファイルとして保存
    plt.close()

# 各列のヒストグラムと箱ひげ図を一つのPDFファイルにまとめる
from matplotlib.backends.backend_pdf import PdfPages

with PdfPages('all_plots.pdf') as pdf:
  for col in df.columns:
    if df[col].dtype in ['int64', 'float64']:
      # ヒストグラム
      plt.figure()
      df[col].hist()
      plt.xlabel(col)
      plt.ylabel('Frequency')
      plt.title(f'Histogram of {col}')
      pdf.savefig()  # PDFに保存
      plt.close()

      # 箱ひげ図
      plt.figure()
      df.boxplot(column=col)
      plt.title(f'Boxplot of {col}')
      pdf.savefig()  # PDFに保存
      plt.close()

  # 散布図
  for i in range(len(numeric_cols)):
    for j in range(i + 1, len(numeric_cols)):
      col1 = numeric_cols[i]
      col2 = numeric_cols[j]
      plt.figure()
      plt.scatter(df[col1], df[col2])
      plt.xlabel(col1)
      plt.ylabel(col2)
      plt.title(f'Scatter plot of {col1} vs {col2}')
      pdf.savefig()  # PDFに保存
      plt.close()

# ファイルをダウンロード
from google.colab import files
files.download('all_plots.pdf')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# カラム名指定で

In [ ]:
import matplotlib.pyplot as plt

# ID列を指定
id_columns = ['ID1', 'ID2']  # ここに対象のID列名をリストとして指定

for col in id_columns:
    if col in df.columns and df[col].dtype in ['int64', 'float64']:  # 数値データかつ指定したID列であるか確認
        plt.figure()
        df[col].hist()
        plt.xlabel(col)
        plt.ylabel('Frequency')
        plt.title(f'Histogram of {col}')
        plt.savefig(f'{col}_histogram.png')  # ヒストグラムを画像ファイルとして保存
        plt.close()

# ファイルをダウンロード
from google.colab import files
files.download('all_plots.pdf')

# 欠損値の数

In [ ]:
nullcnt_sr = df.isnull().sum()
print(nullcnt_sr)


Year_Birth             0
Education              0
Marital_Status         0
Income                 0
Kidhome                0
Teenhome               0
Dt_Customer            0
Recency                0
MntWines               0
MntFruits              0
MntMeatProducts        0
MntFishProducts        0
MntSweetProducts       0
MntGoldProds           0
NumDealsPurchases      0
NumWebPurchases        0
NumCatalogPurchases    0
NumStorePurchases      0
NumWebVisitsMonth      0
AcceptedCmp3           0
AcceptedCmp4           0
AcceptedCmp5           0
AcceptedCmp1           0
AcceptedCmp2           0
Complain               0
Z_CostContact          0
Z_Revenue              0
Response               0
Age                    0
Age_Group              0
dtype: int64


以下colab環境限定osのやつ  
FPDF*installもある*

In [9]:
import matplotlib.pyplot as plt
import os

# fpdfのインストール（必要な場合）
try:
    from fpdf import FPDF
except ModuleNotFoundError:
    !pip install fpdf
    from fpdf import FPDF

# 保存するフォルダの名前
output_folder = 'graphs'
os.makedirs(output_folder, exist_ok=True)

# ID列を指定
id_columns = ['AcceptedCmp1', 'AcceptedCmp2']

for col in id_columns:
    if col in df.columns:
        # 円グラフを作成
        value_counts = df[col].value_counts()
        plt.figure()
        plt.pie(value_counts, labels=value_counts.index, autopct='%1.1f%%', startangle=90)
        plt.axis('equal')
        plt.title(f'Pie chart of {col}')
        plt.savefig(os.path.join(output_folder, f'{col}_pie_chart.png'))
        plt.close()

# PNGファイルをPDFにまとめる
pdf = FPDF()
for file_name in os.listdir(output_folder):
    if file_name.endswith('.png'):
        pdf.add_page()
        pdf.image(os.path.join(output_folder, file_name), x=10, y=10, w=180)  # 画像をPDFに追加

pdf_output_path = os.path.join(output_folder, 'combined_plots.pdf')
pdf.output(pdf_output_path)

# PDFをダウンロード
from google.colab import files
files.download(pdf_output_path)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>